<a href="https://colab.research.google.com/github/letyciamilene/probabilidade-teoremas-limites-e-aplicacoes/blob/main/campeonato_brasileiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import requests
import os
from dotenv import load_dotenv
from google.colab import files


In [6]:
# Carregar váriaveis do env
load_dotenv()

user_agent = os.getenv('USER_AGENT')


In [10]:
# URL para extrair dados
url = "https://esportes.terra.com.br/futebol/brasileiro-serie-a/brasileiro-2009-1-rodada,2008b7a6b1ced310VgnCLD200000bbcceb0aRCRD.html"

headers = {'User-Agent':user_agent}

# Requisição
req = requests.get(url, headers = headers)

if req.status_code == 200:
  print("A requisição foi um sucesso")

  content = req.content

soup = BeautifulSoup(req.text, 'html.parser')

# Todos os links das rodadas
rodada_links = soup.select('div#rodadas ul li a')


A requisição foi um sucesso


In [11]:
rodada_links

[<a href=" http://esportes.terra.com.br/futebol/brasileiro/2009/interna/0,,OI3718379-EI13759,00-Brasileiro+rodada.html " title="2ª rodada">2</a>,
 <a href=" http://esportes.terra.com.br/futebol/brasileiro/2009/interna/0,,OI3718386-EI13759,00-Brasileiro+rodada.html " title="3ª rodada">3</a>,
 <a href=" http://esportes.terra.com.br/futebol/brasileiro/2009/interna/0,,OI3718433-EI13759,00-Brasileiro+rodada.html " title="4ª rodada">4</a>,
 <a href=" http://esportes.terra.com.br/futebol/brasileiro/2009/interna/0,,OI3718434-EI13759,00-Brasileiro+rodada.html " title="5ª rodada">5</a>,
 <a href=" http://esportes.terra.com.br/futebol/brasileiro/2009/interna/0,,OI3718435-EI13759,00-Brasileiro+rodada.html " title="6ª rodada">6</a>,
 <a href=" http://esportes.terra.com.br/futebol/brasileiro/2009/interna/0,,OI3718437-EI13759,00-Brasileiro+rodada.html " title="7ª rodada">7</a>,
 <a href=" http://esportes.terra.com.br/futebol/brasileiro/2009/interna/0,,OI3718451-EI13759,00-Brasileiro+rodada.html " tit

# Extração (webscrapping)

In [12]:
# DataFrame vazio
df_resultados = pd.DataFrame()

for i in range(0,37):
  # URL para extrair dados
  url = rodada_links[i]['href']

  # Requisição
  req = requests.get(url, headers = headers)

  soup = BeautifulSoup(req.text, 'html.parser')

  # Encontrar a tabela
  table = soup.find_all('table')

  table_str = str(table)

  df = pd.read_html(table_str)[0]

  # Resultado da junção de todas as tabelas
  df_resultados = pd.concat([df_resultados, df], ignore_index=True)

In [16]:
df_resultados.head()

,Data,Hora,Local,Mandante,Unnamed: 4,Unnamed: 5,Unnamed: 6,Visitante,Unnamed: 8,Unnamed: 9
0,16/5,18h30,Maracanã,Flamengo,0,x,0,Avaí,NaN,NaN
1,16/5,18h30,Couto Pereira,Coritiba,2,x,4,Santo André,NaN,NaN
2,16/5,18h30,Mineirão,Atlético-MG,2,x,1,Grêmio,NaN,NaN
3,17/5,16h,Morumbi,São Paulo,2,x,2,Atlético-PR,NaN,NaN
4,17/5,16h,Arena Barueri,Barueri,0,x,0,Fluminense,NaN,NaN


In [17]:
# Salvar
df_resultados.to_csv('resultados_campeonato_brasileiro_2009.csv', index=False)

In [23]:
# Salvar em maquina local
files.download('resultados_campeonato_brasileiro_2009.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Tratamento

In [26]:
df_resultados = pd.read_csv('/content/resultados_campeonato_brasileiro_2009.csv')

In [27]:
df_resultados.head()

,Data,Hora,Local,Mandante,Unnamed: 4,Unnamed: 5,Unnamed: 6,Visitante,Unnamed: 8,Unnamed: 9
0,16/5,18h30,Maracanã,Flamengo,0,x,0,Avaí,NaN,NaN
1,16/5,18h30,Couto Pereira,Coritiba,2,x,4,Santo André,NaN,NaN
2,16/5,18h30,Mineirão,Atlético-MG,2,x,1,Grêmio,NaN,NaN
3,17/5,16h,Morumbi,São Paulo,2,x,2,Atlético-PR,NaN,NaN
4,17/5,16h,Arena Barueri,Barueri,0,x,0,Fluminense,NaN,NaN


In [28]:
# Excluir e renomear colunas
df_tratamento = df_resultados.drop(['Local', 'Unnamed: 5', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
df_tratamento = df_tratamento.rename(columns={'Unnamed: 4': 'gols_mandante',
                                              'Unnamed: 6': 'gols_visitante'})
df_tratamento.head()

,Data,Hora,Mandante,gols_mandante,gols_visitante,Visitante
0,16/5,18h30,Flamengo,0,0,Avaí
1,16/5,18h30,Coritiba,2,4,Santo André
2,16/5,18h30,Atlético-MG,2,1,Grêmio
3,17/5,16h,São Paulo,2,2,Atlético-PR
4,17/5,16h,Barueri,0,0,Fluminense


In [29]:
# Renomear times adicionando M para mandante e V para visitantes
df_tratamento['Mandante'] = df_tratamento['Mandante'].astype(str) + '_M'
df_tratamento['Visitante'] = df_tratamento['Visitante'].astype(str) + '_V'

In [31]:
# Resultados que realmente aconteceram (ganhou, perdeu ou empatou)
partidas_oficiais = {}

for i in range(len(df_tratamento)):
    mandante = df_tratamento.loc[i, 'Mandante']
    visitante = df_tratamento.loc[i, 'Visitante']
    gols_mandante = df_tratamento.loc[i, 'gols_mandante']
    gols_visitante = df_tratamento.loc[i, 'gols_visitante']

    if gols_mandante > gols_visitante:
        resultado_mandante = 1  # Mandante ganhou
        resultado_visitante = 0  # Visitante perdeu
    elif gols_mandante < gols_visitante:
        resultado_mandante = 0  # Mandante perdeu
        resultado_visitante = 1  # Visitante ganhou
    else:
        resultado_mandante = 2  # Empate
        resultado_visitante = 2  # Empate

    partidas_oficiais[f'jogo_{i+1}'] = {
        mandante: resultado_mandante,
        visitante: resultado_visitante
    }

print(partidas_oficiais)

{'jogo_1': {'Flamengo_M': 2, 'Avaí_V': 2}, 'jogo_2': {'Coritiba_M': 0, 'Santo André_V': 1}, 'jogo_3': {'Atlético-MG_M': 1, 'Grêmio_V': 0}, 'jogo_4': {'São Paulo_M': 2, 'Atlético-PR_V': 2}, 'jogo_5': {'Barueri_M': 2, 'Fluminense_V': 2}, 'jogo_6': {'Internacional_M': 1, 'Palmeiras_V': 0}, 'jogo_7': {'Santos_M': 2, 'Goiás_V': 2}, 'jogo_8': {'Botafogo_M': 2, 'Corinthians_V': 2}, 'jogo_9': {'Vitória_M': 1, 'Sport_V': 0}, 'jogo_10': {'Náutico_M': 1, 'Cruzeiro_V': 0}, 'jogo_11': {'Corinthians_M': 1, 'Barueri_V': 0}, 'jogo_12': {'Cruzeiro_M': 1, 'Vitória_V': 0}, 'jogo_13': {'Goiás_M': 0, 'Internacional_V': 1}, 'jogo_14': {'Atlético-PR_M': 0, 'Náutico_V': 1}, 'jogo_15': {'Grêmio_M': 1, 'Botafogo_V': 0}, 'jogo_16': {'Fluminense_M': 0, 'Santos_V': 1}, 'jogo_17': {'Palmeiras_M': 2, 'São Paulo_V': 2}, 'jogo_18': {'Santo André_M': 0, 'Flamengo_V': 1}, 'jogo_19': {'Sport_M': 0, 'Atlético-MG_V': 1}, 'jogo_20': {'Avaí_M': 2, 'Coritiba_V': 2}, 'jogo_21': {'Botafogo_M': 2, 'Sport_V': 2}, 'jogo_22': {'Cor

In [33]:
# Resultados simulados a partir do jogo 281 (ganhou, perdeu ou empatou) - dicionário atualizado a medida que forem ocorrendo as simulações
partidas_simuladas = {}

for i in range(280):
    mandante = df_tratamento.loc[i, 'Mandante']
    visitante = df_tratamento.loc[i, 'Visitante']
    gols_mandante = df_tratamento.loc[i, 'gols_mandante']
    gols_visitante = df_tratamento.loc[i, 'gols_visitante']

    if gols_mandante > gols_visitante:
        resultado_mandante = 1  # Mandante ganhou
        resultado_visitante = 0  # Visitante perdeu
    elif gols_mandante < gols_visitante:
        resultado_mandante = 0  # Mandante perdeu
        resultado_visitante = 1  # Visitante ganhou
    else:
        resultado_mandante = 2  # Empate
        resultado_visitante = 2  # Empate

    partidas_simuladas[f'jogo_{i+1}'] = {
        mandante: resultado_mandante,
        visitante: resultado_visitante
    }

print(partidas_simuladas)

{'jogo_1': {'Flamengo_M': 2, 'Avaí_V': 2}, 'jogo_2': {'Coritiba_M': 0, 'Santo André_V': 1}, 'jogo_3': {'Atlético-MG_M': 1, 'Grêmio_V': 0}, 'jogo_4': {'São Paulo_M': 2, 'Atlético-PR_V': 2}, 'jogo_5': {'Barueri_M': 2, 'Fluminense_V': 2}, 'jogo_6': {'Internacional_M': 1, 'Palmeiras_V': 0}, 'jogo_7': {'Santos_M': 2, 'Goiás_V': 2}, 'jogo_8': {'Botafogo_M': 2, 'Corinthians_V': 2}, 'jogo_9': {'Vitória_M': 1, 'Sport_V': 0}, 'jogo_10': {'Náutico_M': 1, 'Cruzeiro_V': 0}, 'jogo_11': {'Corinthians_M': 1, 'Barueri_V': 0}, 'jogo_12': {'Cruzeiro_M': 1, 'Vitória_V': 0}, 'jogo_13': {'Goiás_M': 0, 'Internacional_V': 1}, 'jogo_14': {'Atlético-PR_M': 0, 'Náutico_V': 1}, 'jogo_15': {'Grêmio_M': 1, 'Botafogo_V': 0}, 'jogo_16': {'Fluminense_M': 0, 'Santos_V': 1}, 'jogo_17': {'Palmeiras_M': 2, 'São Paulo_V': 2}, 'jogo_18': {'Santo André_M': 0, 'Flamengo_V': 1}, 'jogo_19': {'Sport_M': 0, 'Atlético-MG_V': 1}, 'jogo_20': {'Avaí_M': 2, 'Coritiba_V': 2}, 'jogo_21': {'Botafogo_M': 2, 'Sport_V': 2}, 'jogo_22': {'Cor

In [35]:
# Funções
def jogos_anteriores(dicionario_jogos, time, jogo_atual):
    aparicoes = 0
    for jogo, times_resultados in dicionario_jogos.items():
        if jogo == jogo_atual:
            break
        if time in times_resultados:
            aparicoes += 1
    return aparicoes

def vitorias_anteriores(partidas, time, jogo_atual):
    vitorias = 0
    for jogo, times_resultados in partidas.items():
        if jogo == jogo_atual:
            break
        if time in times_resultados and times_resultados[time] == 1:
            vitorias += 1
    return vitorias

def derrotas_anteriores(partidas, time, jogo_atual):
  derrotas = 0
  for jogo, times_resultados in partidas.items():
      if jogo == jogo_atual:
          break
      if time in times_resultados and times_resultados[time] == 0:
          derrotas += 1
  return derrotas

def empates_anteriores(partidas, time, jogo_atual):
  empates = 0
  for jogo, times_resultados in partidas.items():
      if jogo == jogo_atual:
          break
      if time in times_resultados and times_resultados[time] == 2:
          empates += 1
  return empates